<a href="https://colab.research.google.com/github/NuttakitDW/kbtg_data/blob/master/XGboost_kbtg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#version 4
data = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/train_data_v4.csv')
test_X = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/kbtg_data/master/test_data_v4.csv')
data.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cd_ex,cc_month_mean,kp_month_mean
0,1,2,9.0,5,20000,2.0,3500.0,44700.0,11175.0,3.500000e+03
1,2,2,3.0,4,106000,104.0,210000.0,65200.0,14385.0,3.500000e+04
2,3,2,1.0,3,29000,70.0,139700.0,0.0,0.0,2.328333e+04
3,4,2,3.0,5,61000,315.0,6076100.0,770000.0,500000.0,1.012683e+06
4,5,2,9.0,4,18000,0.0,0.0,12000.0,6000.0,0.000000e+00


In [3]:
test_X.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cd_ex,cc_month_mean,kp_month_mean
0,50001,1,9.0,3,0.0,28.0,64600.0,31800.0,10600.000000,10766.666667
1,50002,1,9.0,4,0.0,131.0,93700.0,5900.0,2950.000000,15616.666667
2,50003,1,3.0,4,0.0,83.0,202500.0,0.0,0.000000,33750.000000
3,50004,2,3.0,4,0.0,20.0,11000.0,7200.0,1440.000000,1833.333333
4,50005,1,3.0,4,0.0,67.0,195500.0,14800.0,4933.333333,32583.333333


In [4]:
data['overall'] = data['kp_txn_amt'] + data['cd_ex']
data.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cd_ex,cc_month_mean,kp_month_mean,overall
0,1,2,9.0,5,20000,2.0,3500.0,44700.0,11175.0,3.500000e+03,48200.0
1,2,2,3.0,4,106000,104.0,210000.0,65200.0,14385.0,3.500000e+04,275200.0
2,3,2,1.0,3,29000,70.0,139700.0,0.0,0.0,2.328333e+04,139700.0
3,4,2,3.0,5,61000,315.0,6076100.0,770000.0,500000.0,1.012683e+06,6846100.0
4,5,2,9.0,4,18000,0.0,0.0,12000.0,6000.0,0.000000e+00,12000.0


In [0]:
test_X['overall'] = test_X['kp_txn_amt'] + test_X['cd_ex']

In [0]:
y = data.income
X = data.drop(['income'], axis=1)
feature = ['ocp_cd', 'age', 'cd_ex', 'kp_txn_amt', 'kp_txn_count', 'gender', 'cc_month_mean', 'kp_month_mean', 'overall']
X = X[feature]

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

categorical_cols = [cname for cname in ['ocp_cd', 'age', 'gender']]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full.copy()
X_valid = X_valid_full.copy()

In [7]:
X_valid.head()

,ocp_cd,age,cd_ex,kp_txn_amt,kp_txn_count,gender,cc_month_mean,kp_month_mean,overall
11841,9.0,3,4600.0,223100.0,39.0,2,2533.333333,44620.000000,227700.0
19602,4.0,4,273400.0,0.0,0.0,1,45566.666667,0.000000,273400.0
45519,3.0,4,10400.0,0.0,0.0,2,1733.333333,0.000000,10400.0
25747,9.0,5,0.0,0.0,0.0,1,0.000000,0.000000,0.0
42642,4.0,3,1470200.0,4822400.0,228.0,2,245033.333333,803733.333333,6292600.0


In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [0]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_absolute_error


In [0]:
from sklearn.ensemble import RandomForestRegressor
rdf = RandomForestRegressor(n_estimators=700, random_state=1)

In [0]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

In [0]:
parameters = {"learning_rate": [0.1, 0.01, 0.001],
               "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
               "max_depth": [2, 4, 7, 10],
               "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
               "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
               "reg_alpha": [0, 0.5, 1],
               "reg_lambda": [1, 1.5, 2, 3, 4.5],
               "min_child_weight": [1, 3, 5, 7],
               "n_estimators": [100, 250, 500, 1000]}

In [0]:
xgb_rscv = RandomizedSearchCV(xgb_model, param_distributions = parameters, scoring = "neg_mean_absolute_error",
                             cv = 7, verbose = 3, random_state = 40)

In [23]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', xgb_rscv)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

Fitting 7 folds for each of 10 candidates, totalling 70 fits
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 
[14:45:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=-25480.185, total=   2.2s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 
[14:45:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=-24874.650, total=   2.2s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 
[14:45:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[CV]  subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6, score=-36815.499, total=  12.0s
[CV] subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6 
[15:05:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6, score=-37965.659, total=  11.9s
[15:05:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed: 19.7min finished
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


MAE: 23214.138687597657


In [26]:
# Model best estimators
print("Learning Rate: ", xgb_rscv.best_estimator_.get_params()["learning_rate"])
print("Gamma: ", xgb_rscv.best_estimator_.get_params()["gamma"])
print("Max Depth: ", xgb_rscv.best_estimator_.get_params()["max_depth"])
print("Subsample: ", xgb_rscv.best_estimator_.get_params()["subsample"])
print("Max Features at Split: ", xgb_rscv.best_estimator_.get_params()["colsample_bytree"])
print("Alpha: ", xgb_rscv.best_estimator_.get_params()["reg_alpha"])
print("Lamda: ", xgb_rscv.best_estimator_.get_params()["reg_lambda"])
print("Minimum Sum of the Instance Weight Hessian to Make a Child: ",
      xgb_rscv.best_estimator_.get_params()["min_child_weight"])
print("Number of Trees: ", xgb_rscv.best_estimator_.get_params()["n_estimators"])

Learning Rate:  0.01
Gamma:  0.3
Max Depth:  10
Subsample:  0.7
Max Features at Split:  1.0
Alpha:  0
Lamda:  3
Minimum Sum of the Instance Weight Hessian to Make a Child:  7
Number of Trees:  250


Learning Rate:  0.01
Gamma:  0.3
Max Depth:  10
Subsample:  0.7
Max Features at Split:  1.0
Alpha:  0
Lamda:  3
Minimum Sum of the Instance Weight Hessian to Make a Child:  7
Number of Trees:  250



In [0]:
res = my_pipeline.predict(test_X[feature])

In [0]:
X_valid.head()

In [0]:
from google.colab import files

output = pd.DataFrame({'id': test_X['id'],
                       'income': res})


output.to_csv('submission4.csv', index=False)
files.download('submission4.csv')

In [0]:
output.tail()